In [1171]:
import os
import tweepy as tw
import pandas as pd
from geotext import GeoText
import nltk
import spacy
import locationtagger
import matplotlib.pyplot as plt
from branca.element import Figure
import folium


In [1172]:
consumer_key = "L45cVwt1PcimfFnycMhQEHRsg"
consumer_secret = "a2f3dr6xHHukbCzlvzK5j0mDYg3di3aN8EfixXOhzV9dbhMk1F"
access_token = "1533339797187883008-A5uEjCSsKsH4OBeTaO8g0vHaqoI8h6"
access_token_secret = "UKvwBMeTRN6p4f2ogcrQAQT52Fi6eWUhe2pCnEkaJ8UGq"

In [1173]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [1174]:
# Define the search term and the date_since date as variables
search_words = "#wildfire OR #wildfires OR #forestfire OR #bushfire"
date_since = "2018-01-01"

In [1175]:
new_search = search_words + " -filter:retweets"
new_search

'#wildfire OR #wildfires OR #forestfire OR #bushfire -filter:retweets'

In [1176]:
tweets = tw.Cursor(api.search_tweets,
                       q=new_search,
                       lang="en").items(2000)


In [1177]:
all_tweets = [[tweet.text] for tweet in tweets]


In [1178]:
tweet_text = pd.DataFrame(data=all_tweets, 
                    columns=["tweet"])
                    
tweet_text

,tweet
0,"With serious #flooding, terrible #wildfires, #..."
1,#Wildfires have depleted almost all of the #ca...
2,Firefighters tackle forest fires in Spain's A ...
3,Dave. \n\n#wildfires #WeatherUpdate #weather #...
4,On The Frontlines: Battling The Carter Canyon ...
...,...
1995,#Arctic is still on fire. \nThe #Copernicus #S...
1996,"#WILDFIRE Map Shows California, Oregon and Mon..."
1997,"#WILDFIRE Map Shows California, Oregon and Mon..."
1998,#Wildfires are setting off unexploded ordinanc...


In [1179]:
df = tweet_text[tweet_text["tweet"].str.contains("RT") == False]
df

,tweet
0,"With serious #flooding, terrible #wildfires, #..."
1,#Wildfires have depleted almost all of the #ca...
2,Firefighters tackle forest fires in Spain's A ...
3,Dave. \n\n#wildfires #WeatherUpdate #weather #...
4,On The Frontlines: Battling The Carter Canyon ...
...,...
1995,#Arctic is still on fire. \nThe #Copernicus #S...
1996,"#WILDFIRE Map Shows California, Oregon and Mon..."
1997,"#WILDFIRE Map Shows California, Oregon and Mon..."
1998,#Wildfires are setting off unexploded ordinanc...


In [1180]:
df['tweet'] = df['tweet'].str.title()
df

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/3597067682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'] = df['tweet'].str.title()


,tweet
0,"With Serious #Flooding, Terrible #Wildfires, #..."
1,#Wildfires Have Depleted Almost All Of The #Ca...
2,Firefighters Tackle Forest Fires In Spain'S A ...
3,Dave. \n\n#Wildfires #Weatherupdate #Weather #...
4,On The Frontlines: Battling The Carter Canyon ...
...,...
1995,#Arctic Is Still On Fire. \nThe #Copernicus #S...
1996,"#Wildfire Map Shows California, Oregon And Mon..."
1997,"#Wildfire Map Shows California, Oregon And Mon..."
1998,#Wildfires Are Setting Off Unexploded Ordinanc...


In [1181]:

df_countries = pd.DataFrame()


df_regions = pd.DataFrame()


df_cities = pd.DataFrame()


for index, row in df.iterrows():
    countries = row['tweet']
    place_entity = locationtagger.find_locations(text = countries)
    if place_entity.countries:
     df_countries = df_countries.append({'country': place_entity.countries}, ignore_index=True)
    if place_entity.regions:
     df_regions = df_regions.append({'region': place_entity.regions}, ignore_index=True)
    if place_entity.cities:
        for city_name in place_entity.cities:
            df_cities = df_cities.append({'city': city_name}, ignore_index=True)




/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/2338329552.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities = df_cities.append({'city': city_name}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/2338329552.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities = df_cities.append({'city': city_name}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/2338329552.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_countries = df_countries.append({'country': place_entity.countries}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/2338329552.py:19: FutureWarning: The frame.append method

In [1182]:
df_countries

,country
0,[Spain]
1,[Slovenia]
2,[Slovenia]
3,[Germany]
4,[Tunisia]
...,...
95,[Portugal]
96,[France]
97,[France]
98,[Canada]


In [1183]:
df_regions


,region
0,[California]
1,[Colorado]
2,[Idaho]
3,[California]
4,"[California, Oregon]"
...,...
409,[British Columbia]
410,[California]
411,"[North, California]"
412,"[California, Oregon]"


In [1184]:
df_cities

df_cities.drop_duplicates(subset=['city'], keep=False)

,city
3,Dave
4,Austin
8,Driftwood
22,Crooks
23,Hard
...,...
657,Goes
678,Yreka
684,Sale
691,Flagstaff


In [1185]:
from geopy.geocoders import Nominatim

loc = Nominatim(user_agent="cal")
print(loc)


In [1186]:
df_cities_map = pd.DataFrame()

for ind in df_cities.index:
    getloc = loc.geocode(df_cities['city'][ind])
    df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)

df_cities_map=df_cities_map.dropna()
df_cities_map

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_2270/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_citie

,city,latitude,longitude
0,Reserve,39.976148,-95.565059
1,Us,39.783730,-100.445882
2,Spain,39.326068,-4.837979
3,Dave,50.415451,4.896665
4,Austin,30.271129,-97.743700
...,...,...,...
720,North,40.373661,127.087042
721,California,36.701463,-118.755997
722,Oregon,43.979280,-120.737257
723,California,36.701463,-118.755997


In [1202]:
# Creating Basemap
fig3=Figure(width=550,height=350)
m3=folium.Map(location=[-1.286389, 36.817223],tiles='cartodbpositron',zoom_start=1)
fig3.add_child(m3)

loc = 'Forest fire related 2000 tweets all around the world'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m3.get_root().html.add_child(folium.Element(title_html))
#Adding markers to the map
for index, row in df_cities_map.iterrows():
    folium.Marker(location=[row['latitude'],row['longitude']],tooltip=row['city'],
        icon=folium.Icon(color="red", icon_color="white", icon="fire")).add_to(m3)


m3



In [1200]:
from geopy.exc import GeocoderTimedOut

def put_markers(map, df_cities):
    geo_locator = Nominatim(user_agent="ForestFire")
    for index, row in df_cities_map.iterrows():
        if row['city']:
            try:
                location = geo_locator.geocode(row['city'])
            except GeocoderTimedOut:
                continue
            if location:
                folium.Marker([location.latitude, location.longitude],tooltip=row['city'],
        icon=folium.Icon(color="red", icon_color="white", icon="fire")).add_to(map)



In [1199]:

map = folium.Map(location=[0, 0], zoom_start=2)
loc = 'Forest fire related 1000 tweets all around the world'
map.get_root().html.add_child(folium.Element(title_html))
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
location_data = df_cities
put_markers(map, location_data)
map.save("index.html")